## Need for a uniform interface of running models

As described in the official competition page, to win the prize money, a prerequisite is that the code has to be made open-source. In addition, the top 10 submissions/teams will be invited to become co-authors in a scientific paper that involves further stress-testing of their models in a subsequent phase with many other datasets outside Kaggle platform. **To enable such further analyses and re-use of the models by the community, we strongly encourage** the participants to adhere to a code template that we provide through this repository that enables a uniform interface of running models: [https://github.com/uio-bmi/predict-airr](https://github.com/uio-bmi/predict-airr)


Ideally, all the methods can be run in a unified way, e.g.,

`python3 -m submission.main --train_dir /path/to/train_dir --test_dirs /path/to/test_dir_1 /path/to/test_dir_2 --out_dir /path/to/output_dir --n_jobs 4 --device cpu`

## Adhering to code template on Kaggle Notebooks

Those participants who make use of Kaggle resources and Kaggle notebooks to develop and run their code are also strongly encouraged to copy the code template, particularly the `ImmuneStatePredictor` class and any utility functions from the provided code template repository and adhere to the code template to enable a unified way of running different methods at a later stage. In this notebook, we copied the code template below for participants to paste into their respective Kaggle notebooks and edit as needed.

In [1]:
## imports required for the basic code template below.

import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import glob
import sys
import argparse
import gc
import pickle
import hashlib
from collections import defaultdict, Counter
from typing import Iterator, Tuple, Union, List, Dict
from scipy.stats import entropy, skew
from scipy.optimize import minimize
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.base import clone
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
## some utility functions such as data loaders, etc.

def load_data_generator(data_dir: str, metadata_filename='metadata.csv') -> Iterator[
    Union[Tuple[str, pd.DataFrame, bool], Tuple[str, pd.DataFrame]]]:
    """
    A generator to load immune repertoire data.

    This function operates in two modes:
    1.  If metadata is found, it yields data based on the metadata file.
    2.  If metadata is NOT found, it uses glob to find and yield all '.tsv'
        files in the directory.

    Args:
        data_dir (str): The path to the directory containing the data.

    Yields:
        An iterator of tuples. The format depends on the mode:
        - With metadata: (repertoire_id, pd.DataFrame, label_positive)
        - Without metadata: (filename, pd.DataFrame)
    """
    metadata_path = os.path.join(data_dir, metadata_filename)

    if os.path.exists(metadata_path):
        metadata_df = pd.read_csv(metadata_path)
        for row in metadata_df.itertuples(index=False):
            file_path = os.path.join(data_dir, row.filename)
            try:
                repertoire_df = pd.read_csv(file_path, sep='\t')
                yield row.repertoire_id, repertoire_df, row.label_positive
            except FileNotFoundError:
                print(f"Warning: File '{row.filename}' listed in metadata not found. Skipping.")
                continue
    else:
        search_pattern = os.path.join(data_dir, '*.tsv')
        tsv_files = glob.glob(search_pattern)
        for file_path in sorted(tsv_files):
            try:
                filename = os.path.basename(file_path)
                repertoire_df = pd.read_csv(file_path, sep='\t')
                yield filename, repertoire_df
            except Exception as e:
                print(f"Warning: Could not read file '{file_path}'. Error: {e}. Skipping.")
                continue


def load_full_dataset(data_dir: str) -> pd.DataFrame:
    """
    Loads all TSV files from a directory and concatenates them into a single DataFrame.

    This function handles two scenarios:
    1. If metadata.csv exists, it loads data based on the metadata and adds
       'repertoire_id' and 'label_positive' columns.
    2. If metadata.csv does not exist, it loads all .tsv files and adds
       a 'filename' column as an identifier.

    Args:
        data_dir (str): The path to the data directory.

    Returns:
        pd.DataFrame: A single, concatenated DataFrame containing all the data.
    """
    metadata_path = os.path.join(data_dir, 'metadata.csv')
    df_list = []
    data_loader = load_data_generator(data_dir=data_dir)

    if os.path.exists(metadata_path):
        metadata_df = pd.read_csv(metadata_path)
        total_files = len(metadata_df)
        for rep_id, data_df, label in tqdm(data_loader, total=total_files, desc="Loading files"):
            data_df['ID'] = rep_id
            data_df['label_positive'] = label
            df_list.append(data_df)
    else:
        search_pattern = os.path.join(data_dir, '*.tsv')
        total_files = len(glob.glob(search_pattern))
        for filename, data_df in tqdm(data_loader, total=total_files, desc="Loading files"):
            data_df['ID'] = os.path.basename(filename).replace(".tsv", "")
            df_list.append(data_df)

    if not df_list:
        print("Warning: No data files were loaded.")
        return pd.DataFrame()

    full_dataset_df = pd.concat(df_list, ignore_index=True)
    return full_dataset_df


def load_and_encode_kmers(data_dir: str, k: int = 3) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Loading and k-mer encoding of repertoire data.

    Args:
        data_dir: Path to data directory
        k: K-mer length

    Returns:
        Tuple of (encoded_features_df, metadata_df)
        metadata_df always contains 'ID', and 'label_positive' if available
    """
    from collections import Counter

    metadata_path = os.path.join(data_dir, 'metadata.csv')
    data_loader = load_data_generator(data_dir=data_dir)

    repertoire_features = []
    metadata_records = []

    search_pattern = os.path.join(data_dir, '*.tsv')
    total_files = len(glob.glob(search_pattern))

    for item in tqdm(data_loader, total=total_files, desc=f"Encoding {k}-mers"):
        if os.path.exists(metadata_path):
            rep_id, data_df, label = item
        else:
            filename, data_df = item
            rep_id = os.path.basename(filename).replace(".tsv", "")
            label = None

        kmer_counts = Counter()
        for seq in data_df['junction_aa'].dropna():
            for i in range(len(seq) - k + 1):
                kmer_counts[seq[i:i + k]] += 1

        repertoire_features.append({
            'ID': rep_id,
            **kmer_counts
        })

        metadata_record = {'ID': rep_id}
        if label is not None:
            metadata_record['label_positive'] = label
        metadata_records.append(metadata_record)

        del data_df, kmer_counts

    features_df = pd.DataFrame(repertoire_features).fillna(0).set_index('ID')
    features_df.fillna(0)
    metadata_df = pd.DataFrame(metadata_records)

    return features_df, metadata_df


def save_tsv(df: pd.DataFrame, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    df.to_csv(path, sep='\t', index=False)


def get_repertoire_ids(data_dir: str) -> list:
    """
    Retrieves repertoire IDs from the metadata file or filenames in the directory.

    Args:
        data_dir (str): The path to the data directory.

    Returns:
        list: A list of repertoire IDs.
    """
    metadata_path = os.path.join(data_dir, 'metadata.csv')

    if os.path.exists(metadata_path):
        metadata_df = pd.read_csv(metadata_path)
        repertoire_ids = metadata_df['repertoire_id'].tolist()
    else:
        search_pattern = os.path.join(data_dir, '*.tsv')
        tsv_files = glob.glob(search_pattern)
        repertoire_ids = [os.path.basename(f).replace('.tsv', '') for f in sorted(tsv_files)]

    return repertoire_ids


def generate_random_top_sequences_df(n_seq: int = 50000) -> pd.DataFrame:
    """
    Generates a random DataFrame simulating top important sequences.

    Args:
        n_seq (int): Number of sequences to generate.

    Returns:
        pd.DataFrame: A DataFrame with columns 'ID', 'dataset', 'junction_aa', 'v_call', 'j_call'.
    """
    seqs = set()
    while len(seqs) < n_seq:
        seq = ''.join(np.random.choice(list('ACDEFGHIKLMNPQRSTVWY'), size=15))
        seqs.add(seq)
    data = {
        'junction_aa': list(seqs),
        'v_call': ['TRBV20-1'] * n_seq,
        'j_call': ['TRBJ2-7'] * n_seq,
        'importance_score': np.random.rand(n_seq)
    }
    return pd.DataFrame(data)


def validate_dirs_and_files(train_dir: str, test_dirs: List[str], out_dir: str) -> None:
    assert os.path.isdir(train_dir), f"Train directory `{train_dir}` does not exist."
    train_tsvs = glob.glob(os.path.join(train_dir, "*.tsv"))
    assert train_tsvs, f"No .tsv files found in train directory `{train_dir}`."
    metadata_path = os.path.join(train_dir, "metadata.csv")
    assert os.path.isfile(metadata_path), f"`metadata.csv` not found in train directory `{train_dir}`."

    for test_dir in test_dirs:
        assert os.path.isdir(test_dir), f"Test directory `{test_dir}` does not exist."
        test_tsvs = glob.glob(os.path.join(test_dir, "*.tsv"))
        assert test_tsvs, f"No .tsv files found in test directory `{test_dir}`."

    try:
        os.makedirs(out_dir, exist_ok=True)
        test_file = os.path.join(out_dir, "test_write_permission.tmp")
        with open(test_file, "w") as f:
            f.write("test")
        os.remove(test_file)
    except Exception as e:
        print(f"Failed to create or write to output directory `{out_dir}`: {e}")
        sys.exit(1)


def concatenate_output_files(out_dir: str) -> None:
    """
    Concatenates all test predictions and important sequences TSV files from the output directory.

    This function finds all files matching the patterns:
    - *_test_predictions.tsv
    - *_important_sequences.tsv

    and concatenates them to match the expected output format of submissions.csv.

    Args:
        out_dir (str): Path to the output directory containing the TSV files.

    Returns:
        pd.DataFrame: Concatenated DataFrame with predictions followed by important sequences.
                     Columns: ['ID', 'dataset', 'label_positive_probability', 'junction_aa', 'v_call', 'j_call']
    """
    predictions_pattern = os.path.join(out_dir, '*_test_predictions.tsv')
    sequences_pattern = os.path.join(out_dir, '*_important_sequences.tsv')

    predictions_files = sorted(glob.glob(predictions_pattern))
    sequences_files = sorted(glob.glob(sequences_pattern))

    df_list = []

    for pred_file in predictions_files:
        try:
            df = pd.read_csv(pred_file, sep='\t')
            df_list.append(df)
        except Exception as e:
            print(f"Warning: Could not read predictions file '{pred_file}'. Error: {e}. Skipping.")
            continue

    for seq_file in sequences_files:
        try:
            df = pd.read_csv(seq_file, sep='\t')
            df_list.append(df)
        except Exception as e:
            print(f"Warning: Could not read sequences file '{seq_file}'. Error: {e}. Skipping.")
            continue

    if not df_list:
        print("Warning: No output files were found to concatenate.")
        concatenated_df = pd.DataFrame(
            columns=['ID', 'dataset', 'label_positive_probability', 'junction_aa', 'v_call', 'j_call'])
    else:
        concatenated_df = pd.concat(df_list, ignore_index=True)
    submissions_file = os.path.join(out_dir, 'submissions.csv')
    concatenated_df.to_csv(submissions_file, index=False)
    print(f"Concatenated output written to `{submissions_file}`.")


def get_dataset_pairs(train_dir: str, test_dir: str) -> List[Tuple[str, List[str]]]:
    """Returns list of (train_path, [test_paths]) tuples for dataset pairs."""
    test_groups = defaultdict(list)
    for test_name in sorted(os.listdir(test_dir)):
        if test_name.startswith("test_dataset_"):
            base_id = test_name.replace("test_dataset_", "").split("_")[0]
            test_groups[base_id].append(os.path.join(test_dir, test_name))

    pairs = []
    for train_name in sorted(os.listdir(train_dir)):
        if train_name.startswith("train_dataset_"):
            train_id = train_name.replace("train_dataset_", "")
            train_path = os.path.join(train_dir, train_name)
            pairs.append((train_path, test_groups.get(train_id, [])))

    return pairs

In [3]:
## === REPERTOIRE LOADER (must be defined first) ===

class RepertoireLoader:
    """Simple, fast repertoire loader with memory-only caching."""
    
    def __init__(self):
        self._memory_cache = {}
        
    def load_repertoire(self, file_path: str, max_sequences: int = 10000) -> pd.DataFrame:
        """Load repertoire with memory caching."""
        cache_key = f"{file_path}_{max_sequences}"
        
        if cache_key in self._memory_cache:
            return self._memory_cache[cache_key]
        
        # Read TSV - only essential columns for speed
        try:
            df = pd.read_csv(
                file_path, sep='\t',
                usecols=lambda c: c in ['junction_aa', 'v_call', 'j_call', 'templates', 'duplicate_count'],
                low_memory=True
            )
        except:
            df = pd.read_csv(file_path, sep='\t', low_memory=True)
        
        # Downsample if needed
        if len(df) > max_sequences:
            if 'templates' in df.columns:
                df = df.nlargest(max_sequences, 'templates')
            elif 'duplicate_count' in df.columns:
                df = df.nlargest(max_sequences, 'duplicate_count')
            else:
                df = df.head(max_sequences)
        
        self._memory_cache[cache_key] = df
        return df
    
    def clear_cache(self):
        """Clear memory cache."""
        self._memory_cache.clear()
        gc.collect()

# Global loader
_LOADER = None

def get_cache_loader(cache_dir: str = None) -> RepertoireLoader:
    """Get or create global loader."""
    global _LOADER
    if _LOADER is None:
        _LOADER = RepertoireLoader()
    return _LOADER


## === PUBLIC CLONE DATABASE ===

class PublicCloneDatabase:
    """Fast database of discriminative CDR3 sequences."""
    
    def __init__(self, min_occurrence: int = 3, cache_loader=None):
        self.min_occurrence = min_occurrence
        self.sequence_stats = {}
        self.n_pos_reps = 0
        self.n_neg_reps = 0
        self.cache_loader = cache_loader or get_cache_loader()
        
    def fit(self, train_dir: str, max_sequences: int = 10000) -> 'PublicCloneDatabase':
        """Build database - optimized for speed."""
        metadata_path = os.path.join(train_dir, 'metadata.csv')
        metadata = pd.read_csv(metadata_path)
        
        self.n_pos_reps = int((metadata['label_positive'] == 1).sum())
        self.n_neg_reps = int((metadata['label_positive'] == 0).sum())
        total_reps = self.n_pos_reps + self.n_neg_reps
        
        print(f"Building clone database: {self.n_pos_reps} pos, {self.n_neg_reps} neg")
        
        seq_in_pos = Counter()
        seq_in_neg = Counter()
        seq_info = {}
        
        n_files = len(metadata)
        for i, (_, row) in enumerate(metadata.iterrows()):
            file_path = os.path.join(train_dir, row['filename'])
            if (i + 1) % 10 == 0:
                print(f"  {i+1}/{n_files}...")
            
            try:
                rep_df = self.cache_loader.load_repertoire(file_path, max_sequences)
            except:
                continue
            
            rep_df = rep_df.dropna(subset=['junction_aa'])
            seqs = rep_df['junction_aa'].astype(str)
            lengths = seqs.str.len()
            valid_mask = (lengths >= 8) & (lengths <= 25)
            valid_seqs = seqs[valid_mask].unique()
            
            if row['label_positive']:
                seq_in_pos.update(valid_seqs)
            else:
                seq_in_neg.update(valid_seqs)
            
            # Store v/j for first 100 sequences
            if 'v_call' in rep_df.columns:
                for seq in list(valid_seqs)[:100]:
                    if seq not in seq_info:
                        match = rep_df[rep_df['junction_aa'] == seq].iloc[0]
                        seq_info[seq] = {
                            'v_call': str(match.get('v_call', 'unknown')),
                            'j_call': str(match.get('j_call', 'unknown'))
                        }
        
        # Score sequences
        all_seqs = set(seq_in_pos.keys()) | set(seq_in_neg.keys())
        print(f"Scoring {len(all_seqs)} sequences...")
        
        for seq in all_seqs:
            n_pos = seq_in_pos.get(seq, 0)
            n_neg = seq_in_neg.get(seq, 0)
            total = n_pos + n_neg
            
            if total < self.min_occurrence or n_pos == 0:
                continue
            
            pos_rate = n_pos / self.n_pos_reps
            neg_rate = n_neg / self.n_neg_reps if self.n_neg_reps > 0 else 0
            
            if pos_rate <= neg_rate:
                continue
            
            specificity = n_pos / (n_pos + n_neg)
            coverage = n_pos / self.n_pos_reps
            exp_pos = total * (self.n_pos_reps / total_reps)
            chi2 = ((n_pos - exp_pos) ** 2 / (exp_pos + 0.1))
            ensemble_score = 0.4 * specificity + 0.4 * min(coverage * 5, 1) + 0.2 * min(chi2 / 50, 1)
            
            info = seq_info.get(seq, {'v_call': 'unknown', 'j_call': 'unknown'})
            self.sequence_stats[seq] = {
                'ensemble_score': ensemble_score,
                'specificity': specificity,
                'coverage': coverage,
                'n_pos': n_pos,
                'n_neg': n_neg,
                'v_call': info['v_call'],
                'j_call': info['j_call']
            }
        
        print(f"✅ {len(self.sequence_stats)} discriminative sequences")
        return self
    
    def get_enrichment_features(self, rep_df: pd.DataFrame) -> Dict[str, float]:
        """Get enrichment features for a repertoire."""
        default = {'n_public_seqs': 0.0, 'frac_public_seqs': 0.0, 
                   'ensemble_score_mean': 0.0, 'specificity_mean': 0.0}
        
        if not self.sequence_stats:
            return default
        
        unique_seqs = set(rep_df['junction_aa'].dropna().unique())
        public_seqs = unique_seqs & set(self.sequence_stats.keys())
        
        if not public_seqs:
            return default
        
        scores = [self.sequence_stats[s]['ensemble_score'] for s in public_seqs]
        specs = [self.sequence_stats[s]['specificity'] for s in public_seqs]
        
        return {
            'n_public_seqs': float(len(public_seqs)),
            'frac_public_seqs': float(len(public_seqs) / len(unique_seqs)),
            'ensemble_score_mean': float(np.mean(scores)),
            'specificity_mean': float(np.mean(specs))
        }
    
    def get_top_sequences(self, top_k: int = 50000, dataset_name: str = '') -> pd.DataFrame:
        """Get top discriminative sequences."""
        sorted_seqs = sorted(
            self.sequence_stats.items(),
            key=lambda x: x[1]['ensemble_score'],
            reverse=True
        )[:top_k]
        
        records = []
        for i, (seq, stats) in enumerate(sorted_seqs):
            records.append({
                'ID': f'{dataset_name}_seq_{i+1}' if dataset_name else f'seq_{i+1}',
                'dataset': dataset_name,
                'label_positive_probability': -999.0,
                'junction_aa': seq,
                'v_call': stats['v_call'],
                'j_call': stats['j_call']
            })
        
        return pd.DataFrame(records)

print("✅ RepertoireLoader and PublicCloneDatabase defined")

✅ RepertoireLoader and PublicCloneDatabase defined


In [4]:
## === FEATURE EXTRACTION ===

AMINO_ACIDS = list('ACDEFGHIKLMNPQRSTVWY')

def extract_repertoire_features(rep_df: pd.DataFrame, clone_db=None) -> Dict[str, float]:
    """Extract features from a single TCR repertoire - FAST version."""
    features = {}
    
    rep_df = rep_df.dropna(subset=['junction_aa'])
    seqs = rep_df['junction_aa']
    
    if len(seqs) == 0:
        return {'empty': 1.0}
    
    # CDR3 LENGTH (6 features)
    lengths = seqs.str.len()
    features['len_mean'] = float(lengths.mean())
    features['len_std'] = float(lengths.std()) if len(lengths) > 1 else 0.0
    features['len_median'] = float(lengths.median())
    features['len_short_frac'] = float((lengths < 10).mean())
    features['len_long_frac'] = float((lengths > 18).mean())
    features['len_skew'] = float(skew(lengths)) if len(lengths) > 2 else 0.0
    
    # DIVERSITY (4 features)
    seq_counts = seqs.value_counts()
    probs = seq_counts / seq_counts.sum()
    features['entropy'] = float(entropy(probs))
    features['richness'] = float(len(seq_counts) / len(rep_df))
    features['simpson'] = float(1 - np.sum(probs ** 2))
    features['n_unique'] = float(len(seq_counts))
    
    # CLONALITY (4 features)
    sorted_counts = np.sort(seq_counts.values)[::-1]
    total = sorted_counts.sum()
    if total > 0:
        features['top1_frac'] = float(sorted_counts[0] / total)
        features['top10_frac'] = float(sorted_counts[:min(10, len(sorted_counts))].sum() / total)
        features['top100_frac'] = float(sorted_counts[:min(100, len(sorted_counts))].sum() / total)
    else:
        features['top1_frac'] = 0.0
        features['top10_frac'] = 0.0
        features['top100_frac'] = 0.0
    
    # Gini coefficient
    if len(sorted_counts) > 1 and total > 0:
        n = len(sorted_counts)
        features['gini'] = float((2 * np.sum((np.arange(1, n+1) * sorted_counts)) - (n + 1) * total) / (n * total))
    else:
        features['gini'] = 0.0
    
    # V GENE (4 features)
    if 'v_call' in rep_df.columns:
        v_calls = rep_df['v_call'].dropna()
        if len(v_calls) > 0:
            v_counts = v_calls.value_counts(normalize=True)
            features['n_v'] = float(len(v_counts))
            features['v_entropy'] = float(entropy(v_counts))
            features['v_top1'] = float(v_counts.iloc[0])
        else:
            features['n_v'] = 0.0
            features['v_entropy'] = 0.0
            features['v_top1'] = 0.0
    else:
        features['n_v'] = 0.0
        features['v_entropy'] = 0.0
        features['v_top1'] = 0.0
    
    # J GENE (3 features)
    if 'j_call' in rep_df.columns:
        j_calls = rep_df['j_call'].dropna()
        if len(j_calls) > 0:
            j_counts = j_calls.value_counts(normalize=True)
            features['n_j'] = float(len(j_counts))
            features['j_entropy'] = float(entropy(j_counts))
            features['j_top1'] = float(j_counts.iloc[0])
        else:
            features['n_j'] = 0.0
            features['j_entropy'] = 0.0
            features['j_top1'] = 0.0
    else:
        features['n_j'] = 0.0
        features['j_entropy'] = 0.0
        features['j_top1'] = 0.0
    
    # AA COMPOSITION (simplified - 5 features)
    all_aa = ''.join(seqs.astype(str))
    aa_counts = Counter(all_aa)
    total_aa = sum(aa_counts.values())
    if total_aa > 0:
        # Hydrophobic fraction
        hydrophobic = set('AILMFVPG')
        features['hydrophobic_frac'] = sum(aa_counts.get(aa, 0) for aa in hydrophobic) / total_aa
        # Charged fraction
        charged = set('DEKR')
        features['charged_frac'] = sum(aa_counts.get(aa, 0) for aa in charged) / total_aa
        # Polar fraction
        polar = set('STNQCY')
        features['polar_frac'] = sum(aa_counts.get(aa, 0) for aa in polar) / total_aa
        # Aromatic fraction
        aromatic = set('FWY')
        features['aromatic_frac'] = sum(aa_counts.get(aa, 0) for aa in aromatic) / total_aa
        # Cysteine (important for structure)
        features['cys_frac'] = aa_counts.get('C', 0) / total_aa
    else:
        features['hydrophobic_frac'] = 0.0
        features['charged_frac'] = 0.0
        features['polar_frac'] = 0.0
        features['aromatic_frac'] = 0.0
        features['cys_frac'] = 0.0
    
    # ENRICHMENT from clone database
    if clone_db is not None:
        enrich = clone_db.get_enrichment_features(rep_df)
        features.update(enrich)
    
    return features


def extract_all_features(data_dir: str, clone_db=None, cache_loader=None, 
                         max_sequences: int = 10000) -> Tuple[pd.DataFrame, pd.Series]:
    """Extract features for all repertoires - FAST."""
    cache_loader = cache_loader or get_cache_loader()
    metadata_path = os.path.join(data_dir, 'metadata.csv')
    has_labels = os.path.exists(metadata_path)
    
    feature_records = []
    labels = []
    repertoire_ids = []
    
    if has_labels:
        metadata = pd.read_csv(metadata_path)
        n_files = len(metadata)
        print(f"Extracting features from {n_files} repertoires...")
        
        for i, (_, row) in enumerate(metadata.iterrows()):
            if (i + 1) % 20 == 0:
                print(f"  {i+1}/{n_files}...")
            
            file_path = os.path.join(data_dir, row['filename'])
            try:
                rep_df = cache_loader.load_repertoire(file_path, max_sequences)
                features = extract_repertoire_features(rep_df, clone_db)
                feature_records.append(features)
                labels.append(row['label_positive'])
                repertoire_ids.append(row['repertoire_id'])
            except:
                continue
    else:
        tsv_files = sorted(glob.glob(os.path.join(data_dir, '*.tsv')))
        n_files = len(tsv_files)
        print(f"Extracting features from {n_files} test files...")
        
        for i, file_path in enumerate(tsv_files):
            if (i + 1) % 20 == 0:
                print(f"  {i+1}/{n_files}...")
            
            try:
                rep_df = cache_loader.load_repertoire(file_path, max_sequences)
                features = extract_repertoire_features(rep_df, clone_db)
                feature_records.append(features)
                repertoire_ids.append(os.path.basename(file_path).replace('.tsv', ''))
            except:
                continue
    
    print(f"✅ {len(feature_records)} repertoires processed")
    
    X = pd.DataFrame(feature_records)
    X.index = repertoire_ids
    
    if has_labels:
        return X, pd.Series(labels, index=repertoire_ids)
    return X, None

print("✅ Feature extraction defined")

✅ Feature extraction defined


In [5]:
## === MODEL TRAINING (Anti-Overfitting) ===

class SimpleEnsemble:
    """
    Simple, fast ensemble with anti-overfitting configuration.
    Tracks train vs val AUC to detect overfitting.
    """
    
    def __init__(self, n_folds: int = 5):
        self.n_folds = n_folds
        self.models = []
        self.cv_results = []
        self.feature_names = None
        
    def fit(self, X: pd.DataFrame, y: pd.Series) -> 'SimpleEnsemble':
        """Train with CV and overfitting detection."""
        self.feature_names = X.columns.tolist()
        
        print(f"\nTraining ensemble ({self.n_folds}-fold CV)...")
        print(f"Data: {len(X)} samples, {len(self.feature_names)} features")
        print(f"Classes: {int(y.sum())} pos / {int(len(y)-y.sum())} neg")
        
        skf = StratifiedKFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        
        oof_preds = np.zeros(len(y))
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # ANTI-OVERFITTING LightGBM config
            model = lgb.LGBMClassifier(
                n_estimators=1000,
                num_leaves=8,          # Very few leaves
                max_depth=3,           # Very shallow
                learning_rate=0.01,    # Slow learning
                feature_fraction=0.5,  # Feature dropout
                bagging_fraction=0.5,  # Sample dropout
                bagging_freq=1,
                lambda_l1=5.0,         # Strong L1
                lambda_l2=5.0,         # Strong L2
                min_child_samples=20,  # Need more samples per leaf
                random_state=42,
                verbose=-1,
                n_jobs=-1
            )
            
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                callbacks=[lgb.early_stopping(30, verbose=False)]
            )
            
            self.models.append(model)
            
            # Calculate metrics
            train_pred = model.predict_proba(X_train)[:, 1]
            val_pred = model.predict_proba(X_val)[:, 1]
            oof_preds[val_idx] = val_pred
            
            train_auc = roc_auc_score(y_train, train_pred)
            val_auc = roc_auc_score(y_val, val_pred)
            gap = train_auc - val_auc
            
            # Overfitting diagnosis
            if gap > 0.15:
                status = "🔴 SEVERE OVERFIT"
            elif gap > 0.08:
                status = "🟡 MODERATE OVERFIT"
            elif gap > 0.03:
                status = "🟢 MILD OVERFIT"
            else:
                status = "✅ GOOD"
            
            print(f"Fold {fold+1}: Train={train_auc:.4f} Val={val_auc:.4f} Gap={gap:.4f} {status}")
            
            self.cv_results.append({
                'fold': fold + 1,
                'train_auc': train_auc,
                'val_auc': val_auc,
                'overfit_gap': gap,
                'pred_std': val_pred.std()
            })
        
        # Overall metrics
        overall_auc = roc_auc_score(y, oof_preds)
        avg_gap = np.mean([r['overfit_gap'] for r in self.cv_results])
        
        print(f"\n{'='*50}")
        print(f"Overall OOF AUC: {overall_auc:.4f}")
        print(f"Average Overfit Gap: {avg_gap:.4f}")
        
        if avg_gap > 0.15:
            print("⚠️  SEVERE OVERFITTING - consider simpler model")
        elif avg_gap > 0.08:
            print("⚠️  MODERATE OVERFITTING")
        else:
            print("✅ Generalization looks OK")
        
        return self
    
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """Predict with ensemble averaging."""
        # Align features
        for col in self.feature_names:
            if col not in X.columns:
                X[col] = 0.0
        X = X[self.feature_names].fillna(0)
        
        preds = np.zeros(len(X))
        for model in self.models:
            preds += model.predict_proba(X)[:, 1]
        return preds / len(self.models)
    
    def get_cv_results(self) -> pd.DataFrame:
        return pd.DataFrame(self.cv_results)

print("✅ SimpleEnsemble defined")

✅ SimpleEnsemble defined


In [6]:
## === MAIN PREDICTOR CLASS ===

class ImmuneStatePredictor:
    """Fast TCR repertoire classifier with overfitting protection."""
    
    def __init__(self, n_jobs: int = 1, **kwargs):
        self.n_jobs = n_jobs
        self.clone_db = None
        self.ensemble = None
        self.feature_names = None
        self.important_sequences_ = None
        self.train_dataset_name = None
        self.cache_loader = get_cache_loader()
        
        self.n_folds = kwargs.get('n_folds', 5)
        self.max_sequences = kwargs.get('max_sequences', 10000)
    
    def fit(self, train_dir: str) -> 'ImmuneStatePredictor':
        """Train the classifier."""
        self.train_dataset_name = os.path.basename(train_dir)
        print(f"\n{'='*60}")
        print(f"TRAINING: {self.train_dataset_name}")
        print(f"{'='*60}")
        
        # 1. Build clone database
        print("\n[1/3] Building clone database...")
        self.clone_db = PublicCloneDatabase(cache_loader=self.cache_loader)
        self.clone_db.fit(train_dir, self.max_sequences)
        
        # 2. Extract features
        print("\n[2/3] Extracting features...")
        X, y = extract_all_features(
            train_dir, 
            clone_db=self.clone_db,
            cache_loader=self.cache_loader,
            max_sequences=self.max_sequences
        )
        X = X.fillna(0)
        self.feature_names = X.columns.tolist()
        
        # 3. Train ensemble
        print("\n[3/3] Training ensemble...")
        self.ensemble = SimpleEnsemble(n_folds=self.n_folds)
        self.ensemble.fit(X, y)
        
        # Get important sequences
        self.important_sequences_ = self.clone_db.get_top_sequences(
            top_k=50000, 
            dataset_name=self.train_dataset_name
        )
        
        # Cleanup
        self.cache_loader.clear_cache()
        gc.collect()
        
        print(f"\n{'='*60}")
        print("TRAINING COMPLETE")
        print(f"{'='*60}\n")
        return self
    
    def predict_proba(self, test_dir: str) -> pd.DataFrame:
        """Predict probabilities for test data."""
        print(f"Predicting: {os.path.basename(test_dir)}...")
        
        X_test, _ = extract_all_features(
            test_dir,
            clone_db=self.clone_db,
            cache_loader=self.cache_loader,
            max_sequences=self.max_sequences
        )
        X_test = X_test.fillna(0)
        
        preds = self.ensemble.predict(X_test)
        
        test_name = os.path.basename(test_dir)
        result = pd.DataFrame({
            'ID': X_test.index.tolist(),
            'dataset': test_name,
            'label_positive_probability': preds,
            'junction_aa': -999.0,
            'v_call': -999.0,
            'j_call': -999.0
        })
        
        print(f"  Done: range [{preds.min():.3f}, {preds.max():.3f}]")
        return result
    
    def get_cv_results(self) -> pd.DataFrame:
        if self.ensemble:
            return self.ensemble.get_cv_results()
        return None

print("✅ ImmuneStatePredictor defined")

✅ ImmuneStatePredictor defined


In [7]:
## === TEST ON SINGLE DATASET ===

# Kaggle paths
train_dir_base = "/kaggle/input/adaptive-immune-profiling-challenge-2025/train_datasets/train_datasets"
test_dir_base = "/kaggle/input/adaptive-immune-profiling-challenge-2025/test_datasets/test_datasets"
results_dir = "/kaggle/working/results"

# Get dataset pairs
pairs = get_dataset_pairs(train_dir_base, test_dir_base)

if pairs:
    train_dir, test_dirs = pairs[0]  # First dataset only
    print(f"Testing: {os.path.basename(train_dir)}")
    
    predictor = ImmuneStatePredictor(n_jobs=4, n_folds=5, max_sequences=10000)
    predictor.fit(train_dir)
    
    # Show CV results
    cv = predictor.get_cv_results()
    if cv is not None:
        print("\nCV Results:")
        print(cv.to_string(index=False))
else:
    print("No datasets found!")

Testing: train_dataset_1

TRAINING: train_dataset_1

[1/3] Building clone database...
Building clone database: 200 pos, 200 neg
  10/400...
  20/400...
  30/400...
  40/400...
  50/400...
  60/400...
  70/400...
  80/400...
  90/400...
  100/400...
  110/400...
  120/400...
  130/400...
  140/400...
  150/400...
  160/400...
  170/400...
  180/400...
  190/400...
  200/400...
  210/400...
  220/400...
  230/400...
  240/400...
  250/400...
  260/400...
  270/400...
  280/400...
  290/400...
  300/400...
  310/400...
  320/400...
  330/400...
  340/400...
  350/400...
  360/400...
  370/400...
  380/400...
  390/400...
  400/400...
Scoring 3260730 sequences...
✅ 33582 discriminative sequences

[2/3] Extracting features...
Extracting features from 400 repertoires...
  20/400...
  40/400...
  60/400...
  80/400...
  100/400...
  120/400...
  140/400...
  160/400...
  180/400...
  200/400...
  220/400...
  240/400...
  260/400...
  280/400...
  300/400...
  320/400...
  340/400...
  360/40